In [7]:
from chatterbot import ChatBot
chatbot = ChatBot("TestBot")
print("ChatterBot installé avec succès !")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...


ChatterBot installé avec succès !


[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [1]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

# Création du chatbot
chatbot = ChatBot("SimpleBot")

# Entraînement avec des données de base
trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train("chatterbot.corpus.french")  # Entraînement avec le corpus français

# Boucle de discussion
print("Tape 'exit' pour quitter.")
while True:
    user_input = input("Vous: ")
    if user_input.lower() == "exit":
        break
    response = chatbot.get_response(user_input)
    print("Bot:", response)


Training botprofile.yml: [#                   ] 5%

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Training botprofile.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training food.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training trivia.yml: [####################] 100%
Tape 'exit' pour quitter.
Bot: Bonjour, Comment ca va?
Bot: Je consomme de la RAM, et des chiffres binaires.
Bot: Les Royaumes-Unis de Grande Bretagne
Bot: Salut
Bot: Je suis assez jeune selon vos standards.
Bot: Sputnik 1
Bot: D'apres quel astronaute americain, le telescope spatial Hubble, lance en 1990 et place sur orbite basse, est-il nomme?


No value for search_text was available on the provided input


Bot: Quels sont tes centres d'interets


No value for search_text was available on the provided input


Bot: Quels sont tes centres d'interets


In [14]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import nltk

# Forcer le téléchargement des ressources nécessaires
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


# 📌 Initialisation du chatbot
chatbot = ChatBot("F1Bot")

# 📌 Dossier contenant les fichiers CSV
data_folder = "data"  # 🛠️ Change ce chemin selon ton projet

# 📌 Chargement automatique des fichiers CSV depuis les sous-dossiers
csv_files = {}

for root, _, files in os.walk(data_folder):  # 🔍 Recherche dans tous les sous-dossiers
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            csv_files[file] = pd.read_csv(file_path)

# 📌 Liste pour stocker les questions-réponses de base
training_data = []

# 📅 **1️⃣ Générer des questions-réponses basées sur les CSV**
for file, df in csv_files.items():
    try:
        year = next(filter(str.isdigit, file.split("_")), "inconnue")
    except StopIteration:
        year = "inconnue"

    # 🎯 Entraînement basé sur le calendrier des courses
    if "calendrier" in file.lower():
        for _, row in df.iterrows():
            gp = row.get("Grand Prix", "Inconnu")
            circuit = row.get("Circuit", "Inconnu")
            ville = row.get("Ville", "Inconnu")
            pays = row.get("Pays", "Inconnu")
            date = row.get("Date", "Inconnu")
            heure = row.get("Heure", "Inconnu")

            training_data.append(f"Quand a eu lieu le {gp} en {year} ?")
            training_data.append(f"Le {gp} en {year} a eu lieu le {date} à {ville}, {pays}.")

            training_data.append(f"Où s'est déroulé le {gp} en {year} ?")
            training_data.append(f"Le {gp} en {year} s'est déroulé sur le circuit {circuit}, situé à {ville}, {pays}.")

            training_data.append(f"À quelle heure a commencé le {gp} en {year} ?")
            training_data.append(f"Le {gp} en {year} a commencé à {heure}.")

    # 🏆 Entraînement basé sur les classements
    if "classement" in file.lower():
        for i, row in df.iterrows():
            pilote = row.get("Pilote", "Inconnu")
            equipe = row.get("Équipe", "Inconnu")
            points = row.get("Points", "Inconnu")

            if i == 0:  # Premier pilote = Champion du monde
                training_data.append(f"Qui a gagné le championnat en {year} ?")
                training_data.append(f"Le champion du monde en {year} était {pilote} avec {points} points.")

            training_data.append(f"Combien de points a eu {pilote} en {year} ?")
            training_data.append(f"{pilote} a obtenu {points} points en {year}.")

            training_data.append(f"Dans quelle équipe était {pilote} en {year} ?")
            training_data.append(f"{pilote} courait pour {equipe} en {year}.")

# 📌 Entraînement du chatbot
trainer = ListTrainer(chatbot)
trainer.train(training_data)

print("✅ Chatbot F1 entraîné avec succès !")


# 📌 **2️⃣ Moteur de recherche intelligent**
def search_f1_data(question):
    tokens = word_tokenize(question.lower())  # Tokeniser la question
    result = "Désolé, je n'ai pas cette information."

    for file, df in csv_files.items():
        year = next(filter(str.isdigit, file.split("_")), "inconnue")

        if "calendrier" in file.lower() and ("quand" in tokens or "date" in tokens):
            for _, row in df.iterrows():
                if any(word in row.get("Grand Prix", "").lower() for word in tokens):
                    return f"Le {row['Grand Prix']} a eu lieu le {row['Date']} à {row['Ville']}, {row['Pays']}."

        if "classement" in file.lower() and "points" in tokens:
            for _, row in df.iterrows():
                if any(word in row.get("Pilote", "").lower() for word in tokens):
                    return f"{row['Pilote']} a obtenu {row['Points']} points en {year}."

        if "constructeurs" in file.lower() and "constructeur" in tokens:
            for _, row in df.iterrows():
                if any(word in row.get("Équipe", "").lower() for word in tokens):
                    return f"L'équipe {row['Équipe']} a terminé avec {row['Points']} points en {year}."

    return result


# 📌 **3️⃣ Lancer le chatbot interactif**
while True:
    user_input = input("Pose une question sur la F1 : ")
    if user_input.lower() == "exit":
        break

    # 📌 Vérifier si la réponse doit être recherchée dans les fichiers CSV
    response = search_f1_data(user_input)
    if response == "Désolé, je n'ai pas cette information.":
        response = chatbot.get_response(user_input)  # Demander à ChatterBot

    print("F1Bot:", response)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...


List Trainer: [                    ] 1%

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


List Trainer: [####################] 100%
✅ Chatbot F1 entraîné avec succès !


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\cantfly/nltk_data'
    - 'c:\\Users\\cantfly\\AppData\\Local\\Programs\\Python\\Python38\\nltk_data'
    - 'c:\\Users\\cantfly\\AppData\\Local\\Programs\\Python\\Python38\\share\\nltk_data'
    - 'c:\\Users\\cantfly\\AppData\\Local\\Programs\\Python\\Python38\\lib\\nltk_data'
    - 'C:\\Users\\cantfly\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [15]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')  # 📌 Assure que punkt est bien téléchargé

# Test de la tokenisation
question = "Quand a eu lieu le Grand Prix de Monaco ?"
try:
    tokens = word_tokenize(question.lower())
    print("✅ Tokenisation réussie :", tokens)
except Exception as e:
    print("❌ Erreur lors de la tokenisation :", str(e))


❌ Erreur lors de la tokenisation : 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\cantfly/nltk_data'
    - 'c:\\Users\\cantfly\\AppData\\Local\\Programs\\Python\\Python38\\nltk_data'
    - 'c:\\Users\\cantfly\\AppData\\Local\\Programs\\Python\\Python38\\share\\nltk_data'
    - 'c:\\Users\\cantfly\\AppData\\Local\\Programs\\Python\\Python38\\lib\\nltk_data'
    - 'C:\\Users\\cantfly\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
import os
import pandas as pd

data_folder = "data"  # 🛠️ Mets ton chemin ici

# Vérifier si les fichiers sont trouvés
csv_files = {}

for root, _, files in os.walk(data_folder):  
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                csv_files[file] = df
                print(f"✅ Chargé : {file}")
            except Exception as e:
                print(f"❌ Erreur de chargement {file} :", str(e))

# Vérifier le contenu des fichiers chargés
for file, df in csv_files.items():
    print(f"📂 {file} : {df.shape[0]} lignes, {df.shape[1]} colonnes")


✅ Chargé : calendrier_f1_2020.csv
✅ Chargé : calendrier_f1_2021.csv
✅ Chargé : calendrier_f1_2022.csv
✅ Chargé : calendrier_f1_2023.csv
✅ Chargé : calendrier_f1_2024.csv
✅ Chargé : circuits_f1_2020.csv
✅ Chargé : circuits_f1_2021.csv
✅ Chargé : circuits_f1_2022.csv
✅ Chargé : circuits_f1_2023.csv
✅ Chargé : circuits_f1_2024.csv
✅ Chargé : classement_f1_2020.csv
✅ Chargé : classement_f1_2021.csv
✅ Chargé : classement_f1_2022.csv
✅ Chargé : classement_f1_2023.csv
✅ Chargé : classement_f1_2024.csv
✅ Chargé : constructeurs_f1_2020.csv
✅ Chargé : constructeurs_f1_2021.csv
✅ Chargé : constructeurs_f1_2022.csv
✅ Chargé : constructeurs_f1_2023.csv
✅ Chargé : constructeurs_f1_2024.csv
📂 calendrier_f1_2020.csv : 17 lignes, 7 colonnes
📂 calendrier_f1_2021.csv : 22 lignes, 7 colonnes
📂 calendrier_f1_2022.csv : 22 lignes, 7 colonnes
📂 calendrier_f1_2023.csv : 22 lignes, 7 colonnes
📂 calendrier_f1_2024.csv : 24 lignes, 7 colonnes
📂 circuits_f1_2020.csv : 14 lignes, 4 colonnes
📂 circuits_f1_2021.csv 

In [18]:
import nltk
import shutil

# Supprimer tous les fichiers NLTK (si existants)
nltk_data_path = os.path.expanduser('~/nltk_data')
shutil.rmtree(nltk_data_path, ignore_errors=True)

# Télécharger à nouveau les ressources nécessaires
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

print("✅ Téléchargement terminé, essayez à nouveau.")


✅ Téléchargement terminé, essayez à nouveau.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [19]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# Test de tokenisation
question = "Quand a eu lieu le Grand Prix de Monaco ?"
tokens = word_tokenize(question.lower())
print("✅ Tokenisation réussie :", tokens)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cantfly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\cantfly/nltk_data'
    - 'c:\\Users\\cantfly\\AppData\\Local\\Programs\\Python\\Python38\\nltk_data'
    - 'c:\\Users\\cantfly\\AppData\\Local\\Programs\\Python\\Python38\\share\\nltk_data'
    - 'c:\\Users\\cantfly\\AppData\\Local\\Programs\\Python\\Python38\\lib\\nltk_data'
    - 'C:\\Users\\cantfly\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [25]:
import os
import re
import pandas as pd

# 📌 Fonction pour extraire l'année du nom du fichier
def extract_year_from_filename(filename):
    match = re.search(r"(20\d{2})", filename)  # Recherche une année valide (ex: 2023)
    return match.group(1) if match else "inconnue"

# 📌 Dossier contenant les fichiers CSV
data_folder = "data"  # Mets le bon chemin ici

# 📌 Chargement automatique des fichiers de classement uniquement
classement_files = {}

for root, _, files in os.walk(data_folder):  
    for file in files:
        if file.startswith("classement_f1_") and file.endswith(".csv"):
            file_path = os.path.join(root, file)
            year = extract_year_from_filename(file)
            classement_files[year] = pd.read_csv(file_path)

# 📌 Vérifier si les fichiers ont bien été chargés
for year, df in classement_files.items():
    print(f"📂 Chargé : classement F1 {year} → {df.shape[0]} lignes, {df.shape[1]} colonnes")

# 📌 **Moteur de recherche avancé**
def search_f1_data(question):
    tokens = question.lower().split()

    # 📌 Recherche du champion d'une année spécifique
    for year in classement_files.keys():
        if str(year) in tokens and "champion" in tokens:
            df = classement_files[year]
            champion = df.iloc[0]  # 📌 Le premier pilote est le champion
            return f"Le champion du monde en {year} était {champion['Pilote']} avec {champion['Points']} points."

    # 📊 📌 Qui a eu le plus de points en 5 ans ?
    if "plus" in tokens and "points" in tokens and "ans" in tokens:
        top_pilote = None
        max_points = 0
        for year, df in classement_files.items():
            total_points = df.iloc[0]["Points"]  # 📌 Le champion de chaque année
            if total_points > max_points:
                max_points = total_points
                top_pilote = df.iloc[0]["Pilote"]

        return f"Le pilote qui a eu le plus de points ces 5 dernières années est {top_pilote} avec {max_points} points."

    # 🔮 📌 Prédiction du champion de l'année prochaine
    if "année prochaine" in question or "prochain" in question:
        last_year = max(classement_files.keys())  # 📌 Dernière année connue
        df = classement_files[last_year]
        champion = df.iloc[0]  # 📌 Champion de l'année dernière

        # 📌 Faire une estimation de points
        avg_growth = (df.iloc[0]["Points"] - classement_files[str(int(last_year) - 1)].iloc[0]["Points"]) / 2
        predicted_points = df.iloc[0]["Points"] + avg_growth

        return f"Je pense que {champion['Pilote']} sera encore en tête l'année prochaine avec environ {int(predicted_points)} points."

    return "Désolé, je n'ai pas cette information."

# 📌 **Lancer le chatbot interactif**
while True:
    user_input = input("Pose une question sur la F1 : ")
    if user_input.lower() == "exit":
        break

    response = search_f1_data(user_input)
    print("F1Bot:", response)


📂 Chargé : classement F1 2020 → 23 lignes, 5 colonnes
📂 Chargé : classement F1 2021 → 21 lignes, 5 colonnes
📂 Chargé : classement F1 2022 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2023 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2024 → 24 lignes, 5 colonnes
F1Bot: Je pense que Max Verstappen sera encore en tête l'année prochaine avec environ 368 points.
F1Bot: Je pense que Max Verstappen sera encore en tête l'année prochaine avec environ 368 points.
F1Bot: Désolé, je n'ai pas cette information.
F1Bot: Le champion du monde en 2020 était Lewis Hamilton avec 347 points.
F1Bot: Désolé, je n'ai pas cette information.
F1Bot: Désolé, je n'ai pas cette information.
F1Bot: Désolé, je n'ai pas cette information.
F1Bot: Désolé, je n'ai pas cette information.
F1Bot: Désolé, je n'ai pas cette information.


In [28]:
import os
import re
import pandas as pd
import spacy

# Charger le modèle NLP de spaCy
nlp = spacy.load("fr_core_news_sm")

# 📌 Fonction pour extraire l'année du nom du fichier
def extract_year_from_filename(filename):
    match = re.search(r"(20\d{2})", filename)  # Recherche une année (ex: 2023)
    return match.group(1) if match else "inconnue"

# 📌 Dossier contenant les fichiers CSV
data_folder = "data"  # Mets le bon chemin ici

# 📌 Chargement automatique des fichiers de classement uniquement
classement_files = {}

for root, _, files in os.walk(data_folder):  
    for file in files:
        if file.startswith("classement_f1_") and file.endswith(".csv"):
            file_path = os.path.join(root, file)
            year = extract_year_from_filename(file)
            classement_files[year] = pd.read_csv(file_path)

# 📌 Vérifier si les fichiers ont bien été chargés
for year, df in classement_files.items():
    print(f"📂 Chargé : classement F1 {year} → {df.shape[0]} lignes, {df.shape[1]} colonnes")

# 📌 **Moteur de recherche avancé avec NLP**
def search_f1_data(question):
    doc = nlp(question.lower())  # Analyse NLP de la question
    tokens = [token.text for token in doc]

    # 📌 Recherche du champion d'une année spécifique
    for year in classement_files.keys():
        if str(year) in tokens and any(word in question for word in ["champion", "gagné", "vainqueur"]):
            df = classement_files[year]
            champion = df.iloc[0]  # 📌 Le premier pilote est le champion
            return f"Le champion du monde en {year} était {champion['Pilote']} avec {champion['Points']} points."

    # 📊 📌 Qui a eu le plus de points en 5 ans ?
    if "plus" in tokens and "points" in tokens and "ans" in tokens:
        total_points = {}
        for year, df in classement_files.items():
            for _, row in df.iterrows():
                pilote = row["Pilote"]
                points = row["Points"]
                if pilote in total_points:
                    total_points[pilote] += points
                else:
                    total_points[pilote] = points

        top_pilote = max(total_points, key=total_points.get)
        max_points = total_points[top_pilote]

        return f"Le pilote qui a eu le plus de points ces 5 dernières années est {top_pilote} avec {max_points} points."

    # 📉 📌 Qui a eu le moins de points en 5 ans ?
    if "moins" in tokens and "points" in tokens and "ans" in tokens:
        total_points = {}
        for year, df in classement_files.items():
            for _, row in df.iterrows():
                pilote = row["Pilote"]
                points = row["Points"]
                if pilote in total_points:
                    total_points[pilote] += points
                else:
                    total_points[pilote] = points

        worst_pilote = min(total_points, key=total_points.get)
        min_points = total_points[worst_pilote]

        return f"Le pilote qui a eu le moins de points ces 5 dernières années est {worst_pilote} avec {min_points} points."

    # 🔮 📌 Prédiction du champion de l'année prochaine
    if "année prochaine" in question or "prochain" in question or "saison prochaine" in question:
        last_year = max(classement_files.keys())  # 📌 Dernière année connue
        df = classement_files[last_year]
        champion = df.iloc[0]  # 📌 Champion de l'année dernière

        # 📌 Faire une estimation des points
        if str(int(last_year) - 1) in classement_files:
            previous_df = classement_files[str(int(last_year) - 1)]
            prev_champion = previous_df.iloc[0]
            avg_growth = (df.iloc[0]["Points"] - prev_champion["Points"]) / 2
            predicted_points = df.iloc[0]["Points"] + avg_growth
        else:
            predicted_points = df.iloc[0]["Points"]  # Pas assez de données pour une tendance

        return f"Je pense que {champion['Pilote']} sera encore en tête l'année prochaine avec environ {int(predicted_points)} points."

    return "Désolé, je n'ai pas cette information."

# 📌 **Lancer le chatbot interactif**
while True:
    user_input = input("Pose une question sur la F1 : ")
    if user_input.lower() == "exit":
        break

    response = search_f1_data(user_input)
    print("F1Bot:", response)


📂 Chargé : classement F1 2020 → 23 lignes, 5 colonnes
📂 Chargé : classement F1 2021 → 21 lignes, 5 colonnes
📂 Chargé : classement F1 2022 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2023 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2024 → 24 lignes, 5 colonnes
F1Bot: Désolé, je n'ai pas cette information.
F1Bot: Désolé, je n'ai pas cette information.
F1Bot: Le champion du monde en 2021 était Max Verstappen avec 395.5 points.
F1Bot: Désolé, je n'ai pas cette information.


In [29]:
import os
import re
import pandas as pd
from llama_cpp import Llama  # 🚀 Modèle LLM local

# 📌 Charger le modèle LLM (Mistral 7B GGUF)
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_ctx=2048)

# 📌 Fonction pour extraire l'année du fichier CSV
def extract_year_from_filename(filename):
    match = re.search(r"(20\d{2})", filename)  # Recherche une année (ex: 2023)
    return match.group(1) if match else "inconnue"

# 📌 Dossier contenant les fichiers CSV
data_folder = "data"

# 📌 Chargement des fichiers de classement uniquement
classement_files = {}

for root, _, files in os.walk(data_folder):  
    for file in files:
        if file.startswith("classement_f1_") and file.endswith(".csv"):
            file_path = os.path.join(root, file)
            year = extract_year_from_filename(file)
            classement_files[year] = pd.read_csv(file_path)

# 📌 Vérifier que les fichiers sont bien chargés
for year, df in classement_files.items():
    print(f"📂 Chargé : classement F1 {year} → {df.shape[0]} lignes, {df.shape[1]} colonnes")

# 📌 **Moteur de recherche des résultats F1**
def search_f1_data(question):
    question = question.lower()

    # 📌 Recherche du champion d'une année spécifique
    match = re.search(r"(20\d{2})", question)
    if match and "champion" in question:
        year = match.group(1)
        if year in classement_files:
            df = classement_files[year]
            champion = df.iloc[0]  # 📌 Le premier pilote est le champion
            return f"Le champion du monde en {year} était {champion['Pilote']} avec {champion['Points']} points."

    # 📊 📌 Qui a eu le plus de points en 5 ans ?
    if "plus de points" in question:
        total_points = {}
        for year, df in classement_files.items():
            for _, row in df.iterrows():
                pilote = row["Pilote"]
                points = row["Points"]
                if pilote in total_points:
                    total_points[pilote] += points
                else:
                    total_points[pilote] = points

        top_pilote = max(total_points, key=total_points.get)
        max_points = total_points[top_pilote]

        return f"Le pilote qui a eu le plus de points ces 5 dernières années est {top_pilote} avec {max_points} points."

    # 🔮 📌 Prédiction du champion de l'année prochaine
    if "année prochaine" in question or "qui sera champion" in question:
        last_year = max(classement_files.keys())  # 📌 Dernière année connue
        df = classement_files[last_year]
        champion = df.iloc[0]  # 📌 Champion de l'année dernière

        return f"Je pense que {champion['Pilote']} a de grandes chances d’être en tête l’année prochaine."

    return "Désolé, je n'ai pas cette information."

# 📌 **Générer une réponse avec le LLM**
def generate_response(question, search_result):
    prompt = f"""
    Tu es un expert en Formule 1. Un utilisateur te pose la question :
    "{question}"

    Voici l'information brute trouvée dans les fichiers CSV :
    "{search_result}"

    Réponds de manière naturelle et claire.
    """
    output = llm(prompt, max_tokens=256)
    return output["choices"][0]["text"]

# 📌 **Lancer le chatbot interactif**
while True:
    user_input = input("Pose une question sur la F1 : ")
    if user_input.lower() == "exit":
        break

    # 📌 Recherche dans les fichiers CSV
    search_result = search_f1_data(user_input)

    # 📌 Si on a une réponse, demander au LLM de reformuler
    if search_result != "Désolé, je n'ai pas cette information.":
        response = generate_response(user_input, search_result)
    else:
        response = search_result

    print("F1Bot:", response)


llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32  

📂 Chargé : classement F1 2020 → 23 lignes, 5 colonnes
📂 Chargé : classement F1 2021 → 21 lignes, 5 colonnes
📂 Chargé : classement F1 2022 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2023 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2024 → 24 lignes, 5 colonnes
F1Bot: Désolé, je n'ai pas cette information.
F1Bot: Désolé, je n'ai pas cette information.


KeyboardInterrupt: 

In [30]:
from llama_cpp import Llama

# Charger le modèle
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_ctx=2048)

# Test d'un prompt simple
output = llm("Quel est le champion du monde de F1 en 2023 ?", max_tokens=100)

# Afficher la réponse du modèle
print(output["choices"][0]["text"])


llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32  



Le champion du monde de F1 en 2023 est le pilote Max Verstappen, qui court pour l'écurie Red Bull Racing.


In [38]:
import os
import re
import pandas as pd
from llama_cpp import Llama  # 🚀 Modèle LLM local

# 📌 Charger le modèle LLM (Mistral 7B GGUF)
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_ctx=2048)

# 📌 Fonction pour extraire l'année du fichier CSV
def extract_year_from_filename(filename):
    match = re.search(r"(20\d{2})", filename)  # Recherche une année (ex: 2023)
    return match.group(1) if match else "inconnue"

# 📌 Dossier contenant les fichiers CSV
data_folder = "data"

# 📌 Chargement des fichiers de classement uniquement
classement_files = {}

for root, _, files in os.walk(data_folder):  
    for file in files:
        if file.startswith("classement_f1_") and file.endswith(".csv"):
            file_path = os.path.join(root, file)
            year = extract_year_from_filename(file)
            classement_files[year] = pd.read_csv(file_path)

# 📌 Vérifier que les fichiers sont bien chargés
for year, df in classement_files.items():
    print(f"📂 Chargé : classement F1 {year} → {df.shape[0]} lignes, {df.shape[1]} colonnes")

def search_f1_data(question):
    question = question.lower()
    search_result = None

    # 📌 Vérifier s'il y a une année dans la question
    match = re.search(r"(20\d{2})", question)
    year = match.group(1) if match else None

    # 📌 Vérifier si la question concerne le classement
    if "classement" in question and year:
        file_key = f"classement_f1_{year}.csv"
        if year in classement_files:
            df = classement_files[year]
            search_result = df.to_string(index=False)  # Convertir le DataFrame en texte brut

    # 📌 Vérifier si la question concerne le calendrier des courses
    elif "calendrier" in question and year:
        file_key = f"calendrier_f1_{year}.csv"
        calendar_file = os.path.join(data_folder, "calendrier", file_key)
        if os.path.exists(calendar_file):
            df = pd.read_csv(calendar_file)
            search_result = df.to_string(index=False)

    # 📌 Vérifier si la question concerne les circuits
    elif "circuit" in question and year:
        file_key = f"circuits_f1_{year}.csv"
        circuit_file = os.path.join(data_folder, "circuits", file_key)
        if os.path.exists(circuit_file):
            df = pd.read_csv(circuit_file)
            search_result = df.to_string(index=False)

    # 📌 Vérifier si la question concerne les constructeurs
    elif "constructeur" in question and year:
        file_key = f"constructeurs_f1_{year}.csv"
        constructor_file = os.path.join(data_folder, "constructeurs", file_key)
        if os.path.exists(constructor_file):
            df = pd.read_csv(constructor_file)
            search_result = df.to_string(index=False)

    if search_result:
        return f"Voici les données extraites du fichier **{file_key}** :\n\n{search_result}"
    
    print(question)
    print(search_result)

    return "Désolé, je n'ai pas trouvé de données correspondantes."


# 📌 **Générer une réponse avec le LLM**
def generate_response(question, search_result):
    prompt = f"""
    [INST] 
    Tu es un expert en Formule 1. Un utilisateur te pose la question :
    "{question}"

    Voici les données brutes extraites des fichiers CSV :
    {search_result}

    Analyse les données et donne une réponse claire et concise.
    [/INST]
    """

    output = llm(prompt, max_tokens=256)
    return output["choices"][0]["text"]



# 📌 **Lancer le chatbot interactif**
while True:
    user_input = input("Pose une question sur la F1 : ")
    if user_input.lower() == "exit":
        break

    # 🔍 Recherche dans les fichiers CSV
    search_result = search_f1_data(user_input)

    # 🧠 Génération de la réponse avec le LLM
    response = generate_response(user_input, search_result)
    print("F1Bot:", response)



llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32  

📂 Chargé : classement F1 2020 → 23 lignes, 5 colonnes
📂 Chargé : classement F1 2021 → 21 lignes, 5 colonnes
📂 Chargé : classement F1 2022 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2023 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2024 → 24 lignes, 5 colonnes


llama_perf_context_print:        load time =   42682.07 ms
llama_perf_context_print: prompt eval time =   42681.01 ms /   530 tokens (   80.53 ms per token,    12.42 tokens per second)
llama_perf_context_print:        eval time =   39297.90 ms /   255 runs   (  154.11 ms per token,     6.49 tokens per second)
llama_perf_context_print:       total time =   82258.61 ms /   785 tokens


F1Bot: 
    Voici le classement en 2023 de Formule 1 :

    Max Verstappen (Red Bull) - 575 points et 19 victoires
    Sergio Pérez (Red Bull) - 285 points et 2 victoires
    Lewis Hamilton (Mercedes) - 234 points et 0 victoires
    Fernando Alonso (Aston Martin) - 206 points et 0 victoires
    Charles Leclerc (Ferrari) - 206 points et 0 victoires
    Lando Norris (McLaren) - 205 points et 0 victoires
    Carlos Sainz (Ferrari) - 200 points et 1 victoire
    George Russell (Mercedes) - 175 points et 0 victoires
    Oscar Piastri (McLaren) - 97 points et 0 victoires
    Lance Stroll (Aston Martin) - 74 points et 0 victoires
    Pierre Gasly (Alpine F1 Team) - 62


Llama.generate: 37 prefix-match hit, remaining 535 prompt tokens to eval
llama_perf_context_print:        load time =   42682.07 ms
llama_perf_context_print: prompt eval time =   41581.65 ms /   535 tokens (   77.72 ms per token,    12.87 tokens per second)
llama_perf_context_print:        eval time =   22615.78 ms /   147 runs   (  153.85 ms per token,     6.50 tokens per second)
llama_perf_context_print:       total time =   64331.83 ms /   682 tokens


F1Bot: 
    En 2024, Max Verstappen a été le pilote de Red Bull et a remporté 9 courses, obtenu 437 points et terminé 1er du championnat. Lando Norris et George Russell, également du Red Bull, ont fini 2e et 6e du championnat respectivement. Charles Leclerc du Ferrari a terminé 3e avec 356 points, tandis que Pierre Gasly du Red Bull et Nico Hülkenberg de l'Haas F1 Team ont fini 10e et 11e du championnat avec respectivement 42 et 41 points.


Llama.generate: 31 prefix-match hit, remaining 502 prompt tokens to eval
llama_perf_context_print:        load time =   42682.07 ms
llama_perf_context_print: prompt eval time =   38990.30 ms /   502 tokens (   77.67 ms per token,    12.87 tokens per second)
llama_perf_context_print:        eval time =    5127.92 ms /    34 runs   (  150.82 ms per token,     6.63 tokens per second)
llama_perf_context_print:       total time =   44144.55 ms /   536 tokens


F1Bot: 
    Max Verstappen was the winner of the Formula 1 Championship in 2023 with 575 points and 19 victories.


In [ ]:
import os
import re
import pandas as pd
from llama_cpp import Llama  # 🚀 Modèle LLM local

# 📌 Charger le modèle LLM (Mistral 7B GGUF)
llm = Llama(model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_ctx=2048)

# 📌 Fonction pour extraire l'année du fichier CSV
def extract_year_from_filename(filename):
    match = re.search(r"(20\d{2})", filename)  # Recherche une année (ex: 2023)
    return match.group(1) if match else "inconnue"

# 📌 Dossier contenant les fichiers CSV
data_folder = "data"

# 📌 Chargement des fichiers de classement uniquement
classement_files = {}

for root, _, files in os.walk(data_folder):  
    for file in files:
        if file.startswith("classement_f1_") and file.endswith(".csv"):
            file_path = os.path.join(root, file)
            year = extract_year_from_filename(file)
            classement_files[year] = pd.read_csv(file_path)
        if file.startswith("calendrier_f1_") and file.endswith(".csv"):
            file_path = os.path.join(root, file)
            year = extract_year_from_filename(file)
            classement_files[year] = pd.read_csv(file_path)
        if file.startswith("circuits_f1_") and file.endswith(".csv"):
            file_path = os.path.join(root, file)
            year = extract_year_from_filename(file)
            classement_files[year] = pd.read_csv(file_path)
        if file.startswith("constructeurs_f1_") and file.endswith(".csv"):
            file_path = os.path.join(root, file)
            year = extract_year_from_filename(file)
            classement_files[year] = pd.read_csv(file_path)

# 📌 Vérifier que les fichiers sont bien chargés
for year, df in classement_files.items():
    print(f"📂 Chargé : classement F1 {year} → {df.shape[0]} lignes, {df.shape[1]} colonnes")


def search_f1_data(question):
    question = question.lower()
    search_result = None
    files_to_return = []  # Liste des fichiers CSV à retourner

    # 📌 Vérifier s'il y a une année spécifique dans la question
    match = re.search(r"(20\d{2})", question)
    year = int(match.group(1)) if match else None
    

    # 📌 Vérifier si l'utilisateur demande plusieurs années
    if any(keyword in question for keyword in ["5 dernières années", "dernières années", "cinq ans"]):
        if year:
            start_year = max(2020, year - 4)  # Assurer qu'on ne descend pas sous 2020
        else:
            start_year = 2020  # Par défaut, on prend les 5 dernières années disponibles
        year = list(range(start_year, start_year + 5))  # Génère les 5 années demandées
    else:
        years = [year] if year else []  # Sinon, utilise l'année trouvée
    print(year)
    # 📌 Vérifier si la question concerne le classement
    if "classement" in question:
        files_to_return = [f"classement_f1_{y}.csv" for y in year]
        print(files_to_return)

    # 📌 Vérifier si la question concerne le calendrier des courses
    elif "calendrier" in question:
        files_to_return = [f"calendrier_f1_{y}.csv" for y in year]

    # 📌 Vérifier si la question concerne les circuits
    elif "circuit" in question:
        files_to_return = [f"circuits_f1_{y}.csv" for y in year]

    # 📌 Vérifier si la question concerne les constructeurs
    elif "constructeur" in question:
        files_to_return = [f"constructeurs_f1_{y}.csv" for y in year]

    # 📌 Charger et concaténer les fichiers trouvés
    data_results = []
    for file_key in files_to_return:
        file_path = os.path.join(data_folder, file_key)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            data_results.append(f"📂 **Données de {file_key}**\n{df.to_string(index=False)}\n\n")

    # 📌 Retourner les résultats concaténés
    if data_results:
        return "\n".join(data_results)

    return "Désolé, je n'ai pas trouvé de données correspondantes."


# 📌 **Générer une réponse avec le LLM**
def generate_response(question, data_results):
    prompt = f"""
    [INST] 
    Tu es un expert en Formule 1. Un utilisateur te pose la question :
    "{question}"

    Voici les données brutes extraites des fichiers CSV :
    {data_results}

    Analyse les données et donne une réponse claire et concise.
    [/INST]
    """

    output = llm(prompt, max_tokens=256)
    return output["choices"][0]["text"]



# 📌 **Lancer le chatbot interactif**
while True:
    user_input = input("Pose une question sur la F1 : ")
    if user_input.lower() == "exit":
        break

    # 🔍 Recherche dans les fichiers CSV
    data_results = search_f1_data(user_input)

    # 🧠 Génération de la réponse avec le LLM
    response = generate_response(user_input, data_results)
    print("F1Bot:", response)


llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32  

📂 Chargé : classement F1 2020 → 23 lignes, 5 colonnes
📂 Chargé : classement F1 2021 → 21 lignes, 5 colonnes
📂 Chargé : classement F1 2022 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2023 → 22 lignes, 5 colonnes
📂 Chargé : classement F1 2024 → 24 lignes, 5 colonnes
2020


llama_perf_context_print:        load time =    9432.40 ms
llama_perf_context_print: prompt eval time =    9431.36 ms /   114 tokens (   82.73 ms per token,    12.09 tokens per second)
llama_perf_context_print:        eval time =    9513.16 ms /    65 runs   (  146.36 ms per token,     6.83 tokens per second)
llama_perf_context_print:       total time =   18993.85 ms /   179 tokens


F1Bot: 
    Je regrette que je n'ai pas pu trouver les données correspondantes pour vous répondre à votre question.

    Si vous pouvez fournir plus de détails ou si vous avez d'autres questions, je serai heureux de vous aider.
2020


TypeError: 'int' object is not iterable